# **Implementation of a scoring model**
## **Notebook 2/6 - Creation of features & Data assembly**

This notebook is organized as follows:

**0. Set up**
- 0.1 Loading libraries and useful functions
- 0.2 Loading and description of the dataset
    
**1. Creation of features & Data assembly**
- 1.1 APP_TRAIN data
- 1.2 BUREAU data
- 1.3 BUREAU_BALANCE data
- 1.4 PREVIOUS APPLICATION data
- 1.5 CASH Data
- 1.6 CREDIT CARD Data
- 1.7 PAYMENT Data

**2. Handling of anomalies**
        
**3. Data encoding**

**4. Data export**

___
### 0. SETUP

In this first step, the working framework is put in place, that is to say:
- The necessary Python libraries and packages are loaded
- Useful functions are defined
- The dataset is loaded
___

___
#### 0.1 LOADING LIBRARIES AND USEFUL FUNCTIONS

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import pandas as pd
import numpy as np

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
from sys import path
path.append("./Resources/functions")

import helper_functions as hf
import graphical_functions as gf

___
#### 0.2 LOADING AND DESCRIPTION OF THE DATASET

In [5]:
app_train = pd.read_csv("./Resources/datasets/origin/application_train.csv")

bureau_balance = pd.read_csv("./Resources/datasets/origin/bureau_balance.csv")

bureau = pd.read_csv("./Resources/datasets/origin/bureau.csv")

credit = pd.read_csv("./Resources/datasets/origin/credit_card_balance.csv")

installments = pd.read_csv("./Resources/datasets/origin/installments_payments.csv")

cash = pd.read_csv("./Resources/datasets/origin/POS_CASH_balance.csv")

prev_app = pd.read_csv("./Resources/datasets/origin/previous_application.csv")

___
### 1. CREATION OF FEATURES & ASSEMBLY OF DATA
___

___
#### 1.1 APP_TRAIN DATA

Creation of a business feature: 
- The ratio of initial contribution compared to the price of the property
- The annuity/income ratio 
- The job seniority/age ratio

In [6]:
# Self financed on goods price ratio (%)
app_train["SELF_FINANCED_PERCENT"] = (app_train["AMT_GOODS_PRICE"] - app_train["AMT_CREDIT"])/app_train["AMT_GOODS_PRICE"]*100
app_train["SELF_FINANCED_PERCENT"] = app_train["SELF_FINANCED_PERCENT"].map(lambda x: 0 if x<0 else x)

In [7]:
# ANNUITY ON INCOME ratio (%)
app_train["ANNUITY_ON_INCOME"] = app_train["AMT_ANNUITY"] / app_train["AMT_INCOME_TOTAL"] * 100

In [8]:
# Days employed on age ratio (%)
app_train['DAYS_EMPLOYED_PERCENT'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

___
#### 1.2 BUREAU DATA

Creation of the following features:
- Number of previous bank loans

Creation of the following features per loan per customer:
- For each qualitative feature:
    - number
    - average
    - sum
- For each quantitative feature: 
    - number
    - average
    - maximum
    - minimum
    - sum

In [9]:
# Convert types for less memory usage
bureau = hf.convert_types(bureau, print_info=True)

Original Memory Usage: 0.23 gb.
New Memory Usage: 0.1 gb.


In [10]:
# Creation previous_loan_counts
previous_loan_counts = bureau.groupby('SK_ID_CURR', 
                                      as_index=False)['SK_ID_BUREAU']\
                             .count()\
                             .rename(columns = {'SK_ID_BUREAU':\
                                                'previous_loan_counts'})

# Merge previous_loan_counts with train on SK_ID_CURR, left
app_train = app_train.merge(previous_loan_counts, 
                            on = 'SK_ID_CURR', how = 'left')

# fillna(0) with train
app_train['previous_loan_counts'] = app_train['previous_loan_counts']\
                                    .fillna(0)

In [11]:
# Creation of bureau_counts containing for each possible value 
# of the qualitative features 2 new feature : count and 
# normalized count

bureau_counts = hf.agg_categorical(bureau, 
                                     group_var = 'SK_ID_CURR', 
                                     df_name = 'bureau')

In [12]:
# Creation of bureau_agg containing for each possible value
# of the quantitative features 5 new features : count, max,
# mean, min and sum
 
bureau_agg = hf.agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), 
                            group_var = 'SK_ID_CURR', 
                            df_name = 'bureau')

In [13]:
# Insert computed features into training data

# Merge bureau_counts with app_train
app_train = app_train.merge(bureau_counts, on="SK_ID_CURR", how="left")

# Merge bureau_agg with app_train
app_train = app_train.merge(bureau_agg, on="SK_ID_CURR", how="left")

# Deleting missing columns
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.3 BUREAU_BALANCE DATA

Creation of the following features per loan per customer:
- For each qualitative feature:
    - number
    - average
    - sum
- Mathematical features for each quantitative feature per loan per customer: 
    - number
    - average
    - maximum
    - minimum
    - sum

In [14]:
# Convert types for less memory usage
bureau_balance = hf.convert_types(bureau_balance, print_info=True)

Original Memory Usage: 0.66 gb.
New Memory Usage: 0.25 gb.


In [15]:

# Counts of each type of status for each previous loan
bureau_balance_counts = hf.agg_categorical(bureau_balance, 
                                           group_var = 'SK_ID_BUREAU', 
                                           df_name = 'bureau_balance')

# Creation bureau_balance_agg 
# Calculate value count statistics for each `SK_ID_CURR` 
bureau_balance_agg = hf.agg_numeric(bureau_balance, 
                                    group_var = 'SK_ID_BUREAU', 
                                    df_name = 'bureau_balance')

# Creation bureau_by_loan
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, 
                                          right_index = True, 
                                          left_on = 'SK_ID_BUREAU', 
                                          how = 'outer')

# Merge to include the SK_ID_CURR - Possibly several rows per client
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], 
                                      on = 'SK_ID_BUREAU', 
                                      how = 'left')

# Creation bureau_balance_by_client - One row per client
bureau_balance_by_client = hf.agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), 
                                          group_var = 'SK_ID_CURR', 
                                          df_name = 'client')

In [16]:
# Merge bureau_balance_by_client with app_train
app_train = app_train.merge(bureau_balance_by_client, on="SK_ID_CURR", how="left")

# Deleting missing columns
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.4 PREVIOUS APPLICATION DATA

Creation of the following features:
- Mathematical features for each qualitative feature per client: 
    - sum
    - number
    - average
- Mathematical features for each quantitative feature per client: 
    - number
    - average
    - maximum
    - minimum
    - sum

In [17]:
# Convert types from previous
prev_app = hf.convert_types(prev_app, print_info=True)

Original Memory Usage: 0.49 gb.
New Memory Usage: 0.16 gb.


In [18]:
# Creation previous_agg
prev_agg = hf.agg_numeric(prev_app, 'SK_ID_CURR', 'previous')

# Creation previous_counts
prev_counts = hf.agg_categorical(prev_app, 'SK_ID_CURR', 'previous')

In [19]:
# Merge previous_counts with app_train
app_train = app_train.merge(prev_counts, on="SK_ID_CURR", how="left")

# Merge previous_agg with app_train
app_train = app_train.merge(prev_agg, on="SK_ID_CURR", how="left")

# Deleting missing columns
app_train = hf.remove_missing_columns(app_train)

There are 6 columns with greater than 90% missing values.


___
#### 1.5 CASH DATA

Creation of the following features:
- Mathematical features for each qualitative feature per client: 
    - sum
    - number
    - average
- Mathematical features for each quantitative feature per client: 
    - number
    - average
    - maximum
    - minimum
    - sum

In [20]:
# Convert types of cash
cash = hf.convert_types(cash, print_info=True)

Original Memory Usage: 0.64 gb.
New Memory Usage: 0.29 gb.


In [21]:
# Creation cash_by_client
cash_by_client = hf.aggregate_client(cash, 
                                     group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                     df_names = ['cash', 'client'])

# Merge cash_by_client with app_train
app_train = app_train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')


# Deleting missing columns
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.6 CREDIT CARD DATA

Creation of the following features:
- Mathematical features for each qualitative feature per client: 
    - sum
    - number
    - average
- Mathematical features for each quantitative feature per client: 
    - number
    - average
    - maximum
    - minimum
    - sum

In [22]:
# Convert credit types
credit = hf.convert_types(credit, print_info=True)

Original Memory Usage: 0.71 gb.
New Memory Usage: 0.34 gb.


In [23]:
# Creation credit_by_client
credit_by_client = hf.aggregate_client(credit, 
                                       group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                       df_names = ['credit', 'client'])

# Merge credit_by_client with app_train
app_train = app_train.merge(credit_by_client, on="SK_ID_CURR", how="left")

# Deleting missing columns
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
#### 1.7 PAYMENT DATA

Creation of the following features:
- Mathematical features for each qualitative feature per client: 
    - sum
    - number
    - average
- Mathematical features for each quantitative feature per client: 
    - number
    - average
    - maximum
    - minimum
    - sum

In [24]:
# Convert installment types
installments = hf.convert_types(installments, print_info = True)

Original Memory Usage: 0.87 gb.
New Memory Usage: 0.44 gb.


In [25]:
# Creation installments_by_clients
installments_by_client = hf.aggregate_client(installments, 
                                             group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                             df_names = ['installments', 'client'])

# Merge installments_by_clietns with app_train
app_train = app_train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')

# Deleting missing columns
app_train = hf.remove_missing_columns(app_train)

There are 0 columns with greater than 90% missing values.


___
### 2. PROCESSING ANOMALIES

The exploratory analysis revealed an identical anomaly for 18% of clients: a length of service of 365,000 days. 

We will replace this abnormal value with NaN, while retaining the information that these clients had this anomaly.
___

In [26]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

___
### 3. DATA ENCODING

Transformation of qualitative variables into quantitative variables via the use of:
- label encoding for qualitative variables with less than 2 modalities
- one hot encoding for other qualitative variables
___

In [27]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the data
            le.fit(app_train[col])
            # Transform data
            app_train[col] = le.transform(app_train[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [28]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)

___
### 4. DATA EXPORT
___

In [29]:
app_train.to_csv("./Resources/datasets/assembled/full_training_data.csv", index=False, chunksize=500)